Importando libraries

In [1]:
import pandas as pd
import numpy as np
import talib
from funcoes_preco import organize_data, get_efficiency_ratio, get_stats
import dwx_query
from datetime import datetime, timedelta
from bisect import bisect_right
from scipy import stats

Dados de entrada

In [2]:
# files_dir = 'C:/Users/guicr/AppData/Roaming/MetaQuotes/Terminal/'\
#             'D0E8209F77C8CF37AD8BF550E51FF075/MQL5/Files/'
symbol = 'PETR4'
tf_1 = 'M5'
tf_2 = 'M5'
timeframes = [tf_1, tf_2]
strategy_name = f'cci_er_skew_strategy_{symbol}_{tf_1}_{tf_2}'
# start = datetime(2017, 1, 1).timestamp()
# end = datetime(2020, 12, 31).timestamp()

Gravando dados recebidos em um dicionário

In [3]:
data = {timeframe: organize_data(f'data/{symbol}_{timeframe}.csv') 
        for timeframe in timeframes}

Enviando os dados para dataframes

In [4]:
symbol_data = data.copy()

Copiando os dataframes para uma variável de manipulação

In [5]:
start = datetime(2015, 1, 1)
end = datetime(2021, 12, 31)
eqty_table = {timeframe: 
    symbol_data[timeframe].loc[(symbol_data[timeframe]['Date'] >= start) 
                               & (symbol_data[timeframe]['Date'] <= end)].copy()
    for timeframe in timeframes}
for timeframe in timeframes:
    eqty_table[timeframe].reset_index(inplace=True, drop=True)

Calculando o retorno barra a barra

In [6]:
pct_change = {timeframe: eqty_table[timeframe]['Close'].pct_change() 
              for timeframe in timeframes}
for timeframe in timeframes:
    eqty_table[timeframe]['Percent Change'] = pct_change[timeframe]

Triggers

In [7]:
cci_period = 20
cci = talib.CCI(eqty_table[tf_1]['High'], eqty_table[tf_1]['Low'], 
                eqty_table[tf_1]['Close'], timeperiod=cci_period)
eqty_table[tf_1]['CCI'] = cci

In [8]:
std_dev_period = 15
std_dev = get_stats(eqty_table[tf_1]['Close'], std_dev_period)['std']
eqty_table[tf_1]['Standard Deviation'] = std_dev

Filters

In [9]:
er_period = 8
efficiency_ratio = get_efficiency_ratio(eqty_table[tf_2]['Close'], 
                                        timeperiod=er_period)
eqty_table[tf_2]['Efficiency Ratio'] = efficiency_ratio['efficiency_ratio']


c:\Users\guicr\OneDrive\Programming\Projects\AlgoTrading\funcoes_preco.py:88: RuntimeWarning: invalid value encountered in true_divide
  efficiency_ratio = np.divide(net_change,sum_change).tolist()


In [10]:
skew_period = 15
skewness = get_stats(eqty_table[tf_2]['Close'], skew_period)['skewness']
eqty_table[tf_2]['Skewness'] = skewness

Preparação do Dataframe para receber as informações de trade.
Integração entre o Timeframe maior com o menor.

Mudar o tipo da coluna 'Date' para datetime.

In [11]:
for timeframe in timeframes:
    eqty_table[timeframe]['Date'] = pd.to_datetime(eqty_table[timeframe]['Date'])

Colocando um marcador para identificar a correspondência entre o Timeframe 
maior e o menor.

In [12]:
lwr_timeframe_table = eqty_table[tf_1].copy()
hgr_timeframe_table = eqty_table[tf_2].copy()

In [13]:
index_higher_timeframe = []
for row_lwr_timeframe in lwr_timeframe_table.index:
    index = bisect_right(hgr_timeframe_table['Date'], 
                         lwr_timeframe_table.loc[row_lwr_timeframe, 'Date'])
    index_higher_timeframe.append(index-1)

In [14]:
eqty_table[tf_1]['Index Higher Timeframe'] = index_higher_timeframe

Configuração dos sinais de compra e venda.

In [51]:
open_position = False
position = []
pos_type = []

shift = 0
er_1 = 0.1
er_2 = 1

d_top = 0
d_bottom = 0

for row in eqty_table[tf_1].index:
    
    if row < cci_period:
        continue
    
    # # Verificação de Topo Duplo ou Fundo Duplo
    # if d_bottom == 0:
    #     if eqty_table[tf_1].loc[row-1, 'CCI'] < -100 and \
    #         eqty_table[tf_1].loc[row, 'CCI'] > -100:
    #             d_bottom += 1
                
    # if d_top == 0:
    #     if eqty_table[tf_1].loc[row-1, 'CCI'] > 100 and \
    #         eqty_table[tf_1].loc[row, 'CCI'] < 100:
    #             d_top += 1
                
    # if d_bottom > 0 and eqty_table[tf_1].loc[row, 'CCI'] > 0:
    #     d_bottom = 0
    
    # if d_top > 0 and eqty_table[tf_1].loc[row, 'CCI'] < 0:
    #     d_top = 0                

    # Sinal entrada de Compra
    elif not open_position and \
        d_bottom == 0 and \
        eqty_table[tf_1].loc[row-1, 'CCI'] < 0 and \
        eqty_table[tf_1].loc[row, 'CCI'] > 0 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-shift, 
                                'Skewness'] < -0.5 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-shift, 
                                'Skewness'] > -1 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-shift,
                              'Efficiency Ratio'] < er_1 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-1-shift,
                              'Efficiency Ratio'] < er_2:
            open_position = True
            pos_type = 'Buy'
            # Revisar como fazer arredondamento de 5 em 5
            stop_loss = eqty_table[tf_1].loc[row, 'Close'] - \
                (5 * np.ceil((2 * eqty_table[tf_1].loc[row, 'Standard Deviation']) / 5))
            std = eqty_table[tf_1].loc[row, 'Standard Deviation']
            position.append([row, pos_type, eqty_table[tf_1].loc[row+1, 'Open'],
                             std, stop_loss])
    
    # Sinal de Saída da Compra
    elif open_position and \
        pos_type == 'Buy' and \
        eqty_table[tf_1].loc[row-1, 'CCI'] < 100 and \
        eqty_table[tf_1].loc[row, 'CCI'] > 100:
            open_position = False
            pos_type = 'Exit Buy'
            position.append([row, pos_type, eqty_table[tf_1].loc[row, 'Close'],
                             np.nan, np.nan])
    
    elif open_position and \
        pos_type == 'Buy' and \
        eqty_table[tf_1].loc[row-1, 'CCI'] > -100 and \
        eqty_table[tf_1].loc[row, 'CCI'] < -100:
            open_position = False
            pos_type = 'Exit Buy'
            position.append([row, pos_type, eqty_table[tf_1].loc[row, 'Close'],
                             np.nan, np.nan])
    
    elif open_position and \
        eqty_table[tf_1].loc[row, 'Low'] <= stop_loss:
            open_position = False
            pos_type = 'Exit Buy'
            position.append([row, pos_type, stop_loss, np.nan, np.nan])
            
    # Sinal entrada de Venda
    elif not open_position and \
        d_top == 0 and \
        eqty_table[tf_1].loc[row-1, 'CCI'] > 0 and \
        eqty_table[tf_1].loc[row, 'CCI'] < 0 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-shift, 
                                'Skewness'] > 0.5 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-shift, 
                                'Skewness'] < 1 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-shift,
                              'Efficiency Ratio'] < er_1 and \
        eqty_table[tf_2].loc[eqty_table[tf_1]['Index Higher Timeframe'][row]-1-shift,
                              'Efficiency Ratio'] < er_2:
            open_position = True
            pos_type = 'Sell'
            stop_loss = eqty_table[tf_1].loc[row, 'Close'] + \
                (5 * np.ceil((2 * eqty_table[tf_1].loc[row, 'Standard Deviation']) / 5))
            std = eqty_table[tf_1].loc[row, 'Standard Deviation']
            position.append([row, pos_type, eqty_table[tf_1].loc[row+1, 'Open'],
                             std, stop_loss])
            
    # Sinal de Saída da Venda
    elif open_position and \
        pos_type == 'Sell' and \
        eqty_table[tf_1].loc[row-1, 'CCI'] > -100 and \
        eqty_table[tf_1].loc[row, 'CCI'] < -100:
            open_position = False
            pos_type = 'Exit Sell'
            position.append([row, pos_type, eqty_table[tf_1].loc[row, 'Close'],
                             np.nan, np.nan])
    
    elif open_position and \
        pos_type == 'Sell' and \
        eqty_table[tf_1].loc[row-1, 'CCI'] < 100 and \
        eqty_table[tf_1].loc[row, 'CCI'] > 100:
            open_position = False
            pos_type = 'Exit Sell'
            position.append([row, pos_type, eqty_table[tf_1].loc[row, 'Close'],
                             np.nan, np.nan])
            
    elif open_position and \
        eqty_table[tf_1].loc[row, 'High'] >= stop_loss:
            open_position = False
            pos_type = 'Exit Sell'
            position.append([row, pos_type, stop_loss, np.nan, np.nan])
    
    # Cálculo da Máxima Exposição Favorável e Máxima Exposição Desfavorável
    

In [52]:
eqty_table[tf_1]

,Date,Open,High,Low,Close,TickVolume,Volume,Spread,Percent Change,CCI,Standard Deviation,Efficiency Ratio,Skewness,Index Higher Timeframe
0,2015-01-02 10:05:00,8.53,8.53,8.46,8.47,545,515400,1,NaN,NaN,NaN,NaN,NaN,0
1,2015-01-02 10:10:00,8.47,8.48,8.44,8.47,238,334600,1,0.000000,NaN,NaN,NaN,NaN,1
2,2015-01-02 10:15:00,8.47,8.49,8.45,8.47,170,256500,1,0.000000,NaN,NaN,NaN,NaN,2
3,2015-01-02 10:20:00,8.47,8.51,8.47,8.50,226,322400,1,0.003542,NaN,NaN,NaN,NaN,3
4,2015-01-02 10:25:00,8.50,8.50,8.48,8.48,119,73700,1,-0.002353,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156502,2021-12-30 17:30:00,28.47,28.49,28.44,28.46,768,485000,1,-0.000351,-147.582697,0.021746,0.500000,-0.318902,156502
156503,2021-12-30 17:35:00,28.45,28.48,28.43,28.46,796,712100,1,0.000000,-155.538099,0.021250,0.714286,-0.463057,156503
156504,2021-12-30 17:40:00,28.45,28.46,28.44,28.45,527,468800,1,-0.000351,-153.616704,0.023627,0.750000,-0.311012,156504
156505,2021-12-30 17:45:00,28.45,28.55,28.44,28.52,1023,1487900,1,0.002460,10.582011,0.024276,0.230769,-0.289653,156505


Cálculo do retorno.

In [53]:
df = pd.DataFrame(position, columns=['Index', 'Position Type', 'Position', 
                                     'Standard Deviation', 'Stop Loss'])

In [54]:
df

,Index,Position Type,Position,Standard Deviation,Stop Loss
0,752,Sell,7.50,0.028095,12.49
1,766,Exit Sell,7.50,NaN,NaN
2,900,Sell,7.90,0.051880,12.90
3,915,Exit Sell,8.08,NaN,NaN
4,1053,Sell,8.03,0.026128,13.04
...,...,...,...,...,...
805,155590,Exit Sell,29.70,NaN,NaN
806,156112,Sell,28.29,0.017308,33.30
807,156116,Exit Sell,28.38,NaN,NaN
808,156121,Sell,28.31,0.023343,33.30


In [55]:
df2 = df.iloc[::2] 
df2.reset_index(inplace=True, drop=True)

In [56]:
df3 = df.iloc[1::2]
df3.reset_index(inplace=True, drop=True)
df3.rename(columns={'Position': 'Position Exit', 
                    'Position Type': 'Position Type Exit',
                    'Standard Deviation': 'Standard Deviation Exit',
                    'Stop Loss': 'Stop Loss Exit'}, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [57]:
df = pd.concat([df2, df3], axis=1)
df

,Index,Position Type,Position,Standard Deviation,Stop Loss,Index,Position Type Exit,Position Exit,Standard Deviation Exit,Stop Loss Exit
0,752,Sell,7.50,0.028095,12.49,766,Exit Sell,7.50,NaN,NaN
1,900,Sell,7.90,0.051880,12.90,915,Exit Sell,8.08,NaN,NaN
2,1053,Sell,8.03,0.026128,13.04,1055,Exit Sell,8.00,NaN,NaN
3,1407,Sell,8.67,0.019732,13.67,1413,Exit Sell,8.65,NaN,NaN
4,1517,Sell,8.43,0.022804,13.40,1520,Exit Sell,8.39,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
400,155533,Sell,29.06,0.069012,34.06,155539,Exit Sell,29.00,NaN,NaN
401,155570,Sell,29.68,0.076058,34.68,155572,Exit Sell,29.92,NaN,NaN
402,155577,Sell,29.64,0.095754,34.64,155590,Exit Sell,29.70,NaN,NaN
403,156112,Sell,28.29,0.017308,33.30,156116,Exit Sell,28.38,NaN,NaN


In [58]:
points = []
for row in df.index:
    if df.loc[row, 'Position Type'] == 'Buy':
        points = np.append(points, df.loc[row, 'Position Exit'] - df.loc[row, 'Position'])
    else:
        points = np.append(points, [df.loc[row, 'Position'] - df.loc[row, 'Position Exit']])

In [59]:
stds = df['Standard Deviation']

In [60]:
r = points / stds

In [61]:
r.dropna(inplace=True)

In [62]:
r

0      0.000000
1     -3.469531
2      1.148198
3      1.013606
4      1.754116
         ...   
400    0.869413
401   -3.155468
402   -0.626604
403   -5.200006
404    0.428397
Name: Standard Deviation, Length: 405, dtype: float64

Estatísticas do retorno

In [63]:
last_value = 0
win_stk = 0
loss_stk = 0
pos_stk = 0
neg_stk = 0
max_sum_r = 0
sum_ant = 0
dd = []
for value in r:
    if value >= 0 and last_value >= 0:
        win_stk += 1
        if win_stk > pos_stk:
            pos_stk = win_stk
    elif value < 0 and last_value <= 0:
        loss_stk += 1
        if loss_stk > neg_stk:
            neg_stk = loss_stk
    else:
        win_stk = 1
        loss_stk = 1
    
    last_value = value
    
    sum_r = sum_ant + value
    sum_ant = sum_r
    if max_sum_r < sum_r:
        max_sum_r = sum_r
    dd.append(sum_r - max_sum_r)
    
print(pos_stk)
print(neg_stk)
print(abs(min(dd)))   

9
6
73.5911899145998


In [64]:
sum(r)

-32.97395411495894

In [65]:
stats.describe(r)

DescribeResult(nobs=405, minmax=(-15.732132722552526, 9.749005254295161), mean=-0.08141717065421954, variance=7.835954585266805, skewness=-1.3129934013137112, kurtosis=6.404832927011192)

In [66]:
z_scores = stats.zscore(r)
abs_z_scores = np.abs(z_scores)
filtered_r = r[abs_z_scores < 3]
stats.describe(filtered_r)

DescribeResult(nobs=399, minmax=(-8.28868445993311, 8.088897765598956), mean=0.059782334067232774, variance=5.4137044367058165, skewness=-0.14356696966315677, kurtosis=0.4270315814751928)

In [67]:
sum(filtered_r)

23.853151292825892

In [68]:
strategy_data = {
    'tf_1': tf_1,
    'tf_2': tf_2,
    'cci_period': cci_period,
    'std_dev_period': std_dev_period,
    'er_period': er_period,
    'skew_period': skew_period,
    'shift': shift,
    'er_1': er_1,
    'er_2': er_2,
}
filename = f'{strategy_name}_specs.txt'
with open(filename, 'w') as f:
    f.write(str(strategy_data))